# Spline Interpolation of Orders 2 and 3

- The user needs to enter the order of interpolation (2 or 3)
- the user must insert the data points (at least 2) in the format: 

In [2]:
import numpy as np

def spline_interpolation(data, order):
    data = np.array(data)
    x, y = data[:, 0], data[:, 1]
    n = len(x)
    
    if n < 2:
        raise ValueError("At least 2 data points")
    
    if order not in [2, 3]:
        raise ValueError("the order must be 2 or 3")
    
    # Number of polynomial segments
    n_segments = n - 1
    
    if order == 2:
        return quadratic_spline(x, y, n_segments)
    else:
        return cubic_spline(x, y, n_segments)

### Quadratic Splin
For the quadratic spline we use the polynomial S_i(x) = a_i(x-x_i)^2 + b_i(x-x_i) + c_i
To solve the linear equation we need constraints:
- Condition 1: Each polynomial passes through its endpoints (Left and Right).
- Condition 2: First derivatives match at interior points: S"(x)=a_ix_(i+1) + b_i = 2a_(i+1)x_(i+1) + b_(i+1).
- Condition 3: We still need a constraint for the left unknown, we assume a boundary condition, natural boundary condition that puts the second derivative of the first point to 0 => a_0 = 0.
<br>
we Start by creating the coef matrix A and vector b, to solve Ax=b, where x is the column matrix of the coefs we're trying to find, we have 3unknown for each equation. Then we set the conditions already disscussed in the hw.

(Note that the reshape() function transform the result 1D vector into a 2D grid)

In [3]:
def quadratic_spline(x, y, n_segments):
    
    n_coeffs = 3 * n_segments  # 3 coefs per segment (a, b, c)
    A = np.zeros((n_coeffs, n_coeffs))
    b = np.zeros(n_coeffs)
    row = 0
    
    # Condition 1
    for i in range(n_segments):
        # Left end
        A[row, 3*i:3*i+3] = [x[i]**2, x[i], 1]
        b[row] = y[i]
        row += 1
        
        # Right end
        A[row, 3*i:3*i+3] = [x[i+1]**2, x[i+1], 1]
        b[row] = y[i+1]
        row += 1
    
    # Condition 2
    for i in range(n_segments - 1):
        A[row, 3*i:3*i+2] = [2*x[i+1], 1]      # Left derivative
        A[row, 3*(i+1):3*(i+1)+2] = [-2*x[i+1], -1]  # Right derivative
        b[row] = 0
        row += 1
    
    # Condition 3 (BC)
    A[row, 0] = 2 
    b[row] = 0
    
    # Solve the linear system
    coeffs = np.linalg.solve(A, b)
    
    return coeffs.reshape(n_segments, 3)

### Cubic Spline Interpolation
We work in a similar way to the quadratic splines however the conditions slightly change and are more.
- Condition 1: Each polynomial passes through its endpoints
- Condition 2: First derivatives match at interior points 3a_ix_(i+1)^2 + 2b_ix_(i+1) + c_i 
- Condition 3: Second derivatives match at interior points 6a_ix_(i+1) + 2b_i
- Condition 4: Natural boundary conditions (second derivatives = 0 at endpoints)  


In [4]:
def cubic_spline(x, y, n_segments):

    n_coeffs = 4 * n_segments
    A = np.zeros((n_coeffs, n_coeffs))
    b = np.zeros(n_coeffs)
    row = 0
    
    # Condition 1
    for i in range(n_segments):
        # Left
        A[row, 4*i:4*i+4] = [x[i]**3, x[i]**2, x[i], 1]
        b[row] = y[i]
        row += 1
        
        # Right 
        A[row, 4*i:4*i+4] = [x[i+1]**3, x[i+1]**2, x[i+1], 1]
        b[row] = y[i+1]
        row += 1
    
    # Condition 2
    for i in range(n_segments - 1):
        
        A[row, 4*i:4*i+3] = [3*x[i+1]**2, 2*x[i+1], 1]
        A[row, 4*(i+1):4*(i+1)+3] = [-3*x[i+1]**2, -2*x[i+1], -1]
        b[row] = 0
        row += 1
    
    # Condition 3
    for i in range(n_segments - 1):
       
        A[row, 4*i:4*i+2] = [6*x[i+1], 2]
        A[row, 4*(i+1):4*(i+1)+2] = [-6*x[i+1], -2]
        b[row] = 0
        row += 1

    
    #Condition 4 (BC)
    A[row, 0:2] = [6*x[0], 2]
    b[row] = 0
    row += 1
    
    
    A[row, 4*(n_segments-1):4*(n_segments-1)+2] = [6*x[-1], 2]
    b[row] = 0
    
    # Solve 
    coeffs = np.linalg.solve(A, b)
    
   
    return coeffs.reshape(n_segments, 4)

### Testing data


In [5]:
test_data = np.array([[0, 0], [1, 1], [2, 4], [3, 9], [4, 16]])

### Testing Quadratic Spline

In [6]:
print("Quadratic spline coefficients:")
quad_coeffs = spline_interpolation(test_data, 2)
print("Coeffs are [a, b, c] for ax² + bx + c")
for i, coeffs in enumerate(quad_coeffs):
    print(f"Segment {i}: {coeffs}")
print()

Quadratic spline coefficients:
Coeffs are [a, b, c] for ax² + bx + c
Segment 0: [0. 1. 0.]
Segment 1: [ 2. -3.  2.]
Segment 2: [ 5.92118946e-16  5.00000000e+00 -6.00000000e+00]
Segment 3: [ 2. -7. 12.]



### Testing Cubic Spline


In [7]:
print("Cubic spline coefficients:")
cubic_coeffs = spline_interpolation(test_data, 3)
print("Coeffs [a, b, c, d] for ax³ + bx² + cx + d")
for i, coeffs in enumerate(cubic_coeffs):
    print(f"Segment {i}: {coeffs}")

Cubic spline coefficients:
Coeffs [a, b, c, d] for ax³ + bx² + cx + d
Segment 0: [0.42857143 0.         0.57142857 0.        ]
Segment 1: [-0.14285714  1.71428571 -1.14285714  0.57142857]
Segment 2: [ 1.42857143e-01 -7.40148683e-15  2.28571429e+00 -1.71428571e+00]
Segment 3: [ -0.42857143   5.14285714 -13.14285714  13.71428571]
